In [1]:
!jt -t l

Didn't recognize theme name: l
Available Themes: 
   chesterish
   grade3
   gruvboxd
   gruvboxl
   monokai
   oceans16
   onedork
   solarizedd
   solarizedl


In [1]:
import requests, json, pandas as pd, numpy as np

## Get medata from HAL

In [2]:
hal_req = """
https://api.archives-ouvertes.fr/search/?q=fulltext_t:\
(("applied a CC BY public copyright") OR ("applied a CC-BY public copyright")\
OR ("appliqué une licence CC BY") OR ("appliqué une licence CC-BY") )\
&rows=5000&sort=submittedDate_tdate desc&\
fl=halId_s,doiId_s,docType_s,primaryDomain_s,journalIssn_s,submittedDate_s,europeanProjectAcronym_s,anrProjectAcronym_s,journalPublisher_s,publisher_s
"""

In [3]:
req_raw = requests.get(hal_req)
req_json = req_raw.json()
print("total of publications", req_json["response"]["numFound"])

total of publications 78


In [4]:
## transform json to dataframe
req_data = req_json["response"]["docs"]
df = pd.DataFrame(req_data)
df.sample(1)

,primaryDomain_s,anrProjectAcronym_s,halId_s,docType_s,submittedDate_s,journalPublisher_s,journalIssn_s,doiId_s,europeanProjectAcronym_s,publisher_s
66,sdv,NaN,hal-03365863,ART,2021-10-05 14:13:08,Rockefeller University Press,0021-9525,10.1083/jcb.202010020,NaN,NaN


In [5]:
## view doctype distribution
df["docType_s"].value_counts()

ART            67
UNDEFINED       8
THESE           1
COMM            1
PROCEEDINGS     1
Name: docType_s, dtype: int64

## Enrich with unpaywall

In [6]:
def get_upw(doi):
    # memo https://api.unpaywall.org/v2/10.1101/2021.05.24.445406?email=maxence.larrieu@u-paris.fr
    req = requests.get(f"https://api.unpaywall.org/v2/{doi}?email=maxence.larrieu@u-paris.fr")
    res = req.json()
    
    # get issns
    issns = res.get("journal_issns")
    
    # deduce if vor is oa
    vor_is_oa = False
    if res.get("best_oa_location"):
        best_loc = res["best_oa_location"]
        vor_is_oa = True if best_loc["version"] == "publishedVersion" else False
    
    return [issns, vor_is_oa]

In [7]:
## pour tester
# memo closed doi 10.1017/S135577181900030X
get_upw("10.1017/S135577181900030X")

['1355-7718,1469-8153', False]

In [8]:
## add upw_issn & upw_vor_is_oa to table
df[["upw_issns", "upw_vor_is_oa" ]] = df.apply(
    lambda row : get_upw(row["doiId_s"]), axis = 1, result_type = "expand")

# Reorder dataframe columns to prepare HTML display

In [9]:
## shorter submittedDAte (no hours info)
### 2022-12-12 is 10 carac length
df["submittedDate_s"] = df["submittedDate_s"].str.slice(0,10)
df.sample(2)

,primaryDomain_s,anrProjectAcronym_s,halId_s,docType_s,submittedDate_s,journalPublisher_s,journalIssn_s,doiId_s,europeanProjectAcronym_s,publisher_s,upw_issns,upw_vor_is_oa
20,sdv,NaN,hal-03837142,ART,2022-11-03,Elsevier,2666-5247,10.1016/S2666-5247(21)00301-3,NaN,NaN,2666-5247,True
36,sdv,NaN,pasteur-03699210,ART,2022-06-20,ACS Publications,2470-1343,10.1021/acsomega.2c01099,NaN,NaN,"2470-1343,2470-1343",False


In [10]:
## make european & anrproject columns boolean (True if data else False)
df["europeanProjectAcronym_s"] = df["europeanProjectAcronym_s"].notna()
df["anrProjectAcronym_s"] = df["anrProjectAcronym_s"].notna()

In [11]:
def deduce_funders(row) : 
    """
    create a column to indicate the funder, either ANR or Europe or both
    """
    if row["europeanProjectAcronym_s"] & row["anrProjectAcronym_s"] : 
        return "Europe & ANR"
    
    elif row["europeanProjectAcronym_s"] : 
        return "Europe"
    
    elif row["anrProjectAcronym_s"] : 
        return "ANR"

In [12]:
df["funded_project"] = df.apply(lambda row : deduce_funders(row), axis = 1)

In [13]:
## make simpler document types
doctype_matcher = {
     "ART" : "article", 
     "UNDEFINED" : "preprint",
     "PROCEEDINGS" : "proceedings",
     "OUV" : "book",
     "COUV" : "book", 
     "ISSUE" : "issue"
    }

In [14]:
def change_doctype(hal_doctype) :
    """
    change document types for less information
    """
    if hal_doctype in doctype_matcher.keys() : 
        return doctype_matcher[hal_doctype]
    else : 
        return "other"

In [15]:
df["docType_s"] = df["docType_s"].apply(lambda x : change_doctype(x))

In [16]:
df.docType_s.value_counts()

article        67
preprint        8
other           2
proceedings     1
Name: docType_s, dtype: int64

In [17]:
df.columns

Index(['primaryDomain_s', 'anrProjectAcronym_s', 'halId_s', 'docType_s',
       'submittedDate_s', 'journalPublisher_s', 'journalIssn_s', 'doiId_s',
       'europeanProjectAcronym_s', 'publisher_s', 'upw_issns', 'upw_vor_is_oa',
       'funded_project'],
      dtype='object')

In [18]:
## remove columns
# /!\/!\/!\ first verify column existance /!\/!\/!\
df_export = df.drop(
    ["primaryDomain_s", "journalIssn_s", "doiId_s", "upw_issns", "anrProjectAcronym_s", "europeanProjectAcronym_s" ], 
    axis = 1)

In [19]:
df_export.columns

Index(['halId_s', 'docType_s', 'submittedDate_s', 'journalPublisher_s',
       'publisher_s', 'upw_vor_is_oa', 'funded_project'],
      dtype='object')

In [20]:
## reorder column
df_export = df[["halId_s", "docType_s", "journalPublisher_s", "upw_vor_is_oa", "funded_project", "submittedDate_s"]]

In [21]:
## rename columns
df_export = df_export.rename(columns = 
                 {'halId_s':'HAL URI', 
                  'docType_s':'genre', 
                  "journalPublisher_s" : "publisher",
                  "upw_vor_is_oa" : "publisher vers.\nis OA",
                  "funded_project" : "European/ANR project",
                  "submittedDate_s" : "deposit date"
                 })

In [22]:
df_export.to_csv("../fr-publications-rrs-applied.csv", index = False)